In [27]:
from pymongo import MongoClient
from pprint import pprint
import datetime
import re

In [28]:
host= "rhea.isegi.unl.pt"
port="27017"
database = "worldwideimporters"
user="mongo_group_14"
password="249774l78m84D1507a9GY9410sy08U83"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}/{database}")

In [29]:
db = client.worldwideimporters
print(f"Database info: {db}\n")

Database info: Database(MongoClient(host=['rhea.isegi.unl.pt:27017'], document_class=dict, tz_aware=False, connect=True), 'worldwideimporters')



In [30]:
collection_list = db.list_collection_names()
print(f"The database contains {len(collection_list)} collections")

print(f"First 5 collections: {collection_list}")

The database contains 3 collections
First 5 collections: ['customers', 'orders', 'people']


In [31]:
# Useful code snipets Resources:

import datetime

d = datetime.datetime(2021, 5, 5, 0, 0, 0, 0)

list(db.collection2.find({"last_modified":{"$lt": d}} ,{"name":1, "last_modified":1}).limit(5))

[]

Authors: 

* Maikel Sousa (m20200735)
* Catarina Moreira (m20201034)
* Luisa Barral (m20201045)
* Xavier Goncalves (m20201090)


# Answering the questions:

0. __Example Question__ _How many orders exist in the database?_

In [32]:
print(f"The database contains {db.orders.count_documents({})} orders.")

The database contains 73595 orders.


__1. How many people records don't have the UserPreferences field?__

In [33]:
records = db.people.count_documents( { "UserPreferences" : { "$exists": False } } )

records

print(f"The people collection has {records} without the UserPreferences field.")

The people collection has 929 without the UserPreferences field.


__2. How many customer records are valid after `November 2015`?__

In [34]:
# Taking the day before the first day of November 2015 to use the greater operator 
dateNov2015 = datetime.datetime(2015, 10, 31, 0, 0, 0, 0) # interval  01/11/2015 - now

valid_cust = db.customers.count_documents({"ValidFrom":{"$gt": dateNov2015}})

print(f"After November 2015, the database has {valid_cust} valid customers.")

After November 2015, the database has 9 valid customers.


__3. How many people have their `Title` equal to `Team Member`?__

In [35]:
# Using exploration to query the correct field:
list(db.people.find({ 'CustomFields.Title': {'$eq':'Team Member'} }).limit(3))

[{'_id': ObjectId('6091cbcf45ad05f8e5c847ba'),
  'PersonID': 2,
  'FullName': 'Kayla Woodcock',
  'PreferredName': 'Kayla',
  'SearchName': 'Kayla Kayla Woodcock',
  'IsPermittedToLogon': True,
  'LogonName': 'kaylaw@wideworldimporters.com',
  'IsExternalLogonProvider': False,
  'HashedPassword': '0x616E9B558976525E7F14D780EBAE80C68586958DC97C506DB418E2E2C49E340E',
  'IsSystemUser': True,
  'IsEmployee': True,
  'IsSalesperson': True,
  'UserPreferences': '{"theme":"humanity","dateFormat":"dd/mm/yy","timeZone": "PST","table":{"pagingType":"full","pageLength": 50},"favoritesOnDashboard":true}',
  'PhoneNumber': '(415) 555-0102',
  'FaxNumber': '(415) 555-0103',
  'EmailAddress': 'kaylaw@wideworldimporters.com',
  'Photo': nan,
  'CustomFields': {'OtherLanguages': ['Polish', 'Chinese', 'Japanese'],
   'HireDate': '2008-04-19T00:00:00',
   'Title': 'Team Member',
   'PrimarySalesTerritory': 'Plains',
   'CommissionRate': '0.98'},
  'OtherLanguages': ['Polish', 'Chinese', 'Japanese'],
  'L

In [36]:
list(db.people.find().limit(3))[1]['CustomFields']

{'OtherLanguages': ['Polish', 'Chinese', 'Japanese'],
 'HireDate': '2008-04-19T00:00:00',
 'Title': 'Team Member',
 'PrimarySalesTerritory': 'Plains',
 'CommissionRate': '0.98'}

In [37]:
# Counting results:
teamMembers = db.people.count_documents({ 'CustomFields.Title': {'$eq':'Team Member'} })

print(f"There are {teamMembers} people with the title Team Member.")

There are 13 people with the title Team Member.


__4. How many people have in their name the string `Sara`?__

In [38]:
# Using exploration to query the correct field:
searchStr = re.compile(r'Sara')

list(db.people.find({"FullName": {'$regex': searchStr}},{"FullName"}).limit(5))

[{'_id': ObjectId('6091cbcf45ad05f8e5c847e0'), 'FullName': 'Sara Karlsson'},
 {'_id': ObjectId('6091cbcf45ad05f8e5c8495f'), 'FullName': 'Sara Charlton'},
 {'_id': ObjectId('6091cbcf45ad05f8e5c84ace'),
  'FullName': 'Saraswati Beniwal'},
 {'_id': ObjectId('6091cbcf45ad05f8e5c84b73'), 'FullName': 'Sara Huiting'},
 {'_id': ObjectId('6091cbcf45ad05f8e5c84bb0'), 'FullName': 'Sara Walkky'}]

We see that the results returned above include the substring `Sara` with the letter S capped and is including a substring next to it. For example, Saraswati.

In [39]:
# Counting results:
SaraCount = db.people.count_documents( { "FullName": {'$regex': searchStr}} )

print(f"There are {SaraCount} people have in their name the string Sara.")

There are 5 people have in their name the string Sara.


__5. Return 5 full names that have in their name the string `Sara`?__

In [40]:
SaraNames = list(db.people.find({"FullName": {'$regex': searchStr}}, {"FullName":1, "_id":0}).limit(5))

SaraNames

[{'FullName': 'Sara Karlsson'},
 {'FullName': 'Sara Charlton'},
 {'FullName': 'Saraswati Beniwal'},
 {'FullName': 'Sara Huiting'},
 {'FullName': 'Sara Walkky'}]

In [41]:
print('We have the following people with their fullnames having the string Sara:\n')

for name in SaraNames:
    print('     *', name['FullName'])

We have the following people with their fullnames having the string Sara:

     * Sara Karlsson
     * Sara Charlton
     * Saraswati Beniwal
     * Sara Huiting
     * Sara Walkky


__6. What is the highest `CommissionRate` that a person has?__

In [42]:
comission = list(db.people.find().sort("CustomFields.CommissionRate", -1).limit(1))
comission

[{'_id': ObjectId('6091cbcf45ad05f8e5c847be'),
  'PersonID': 6,
  'FullName': 'Sophia Hinton',
  'PreferredName': 'Sophia',
  'SearchName': 'Sophia Sophia Hinton',
  'IsPermittedToLogon': True,
  'LogonName': 'sophiah@wideworldimporters.com',
  'IsExternalLogonProvider': False,
  'HashedPassword': '0x451BB10A515F06331540DB392031F9D9BC4EF536A1F86D1CA6C7394556BAA3C0',
  'IsSystemUser': True,
  'IsEmployee': True,
  'IsSalesperson': True,
  'UserPreferences': '{"theme":"black-tie","dateFormat":"mm/dd/yy","timeZone": "PST","table":{"pagingType":"full_numbers","pageLength": 25},"favoritesOnDashboard":true}',
  'PhoneNumber': '(415) 555-0102',
  'FaxNumber': '(415) 555-0103',
  'EmailAddress': 'sophiah@wideworldimporters.com',
  'Photo': nan,
  'CustomFields': {'OtherLanguages': ['Swedish'],
   'HireDate': '2007-05-14T00:00:00',
   'Title': 'Team Member',
   'PrimarySalesTerritory': 'Southeast',
   'CommissionRate': '4.55'},
  'OtherLanguages': ['Swedish'],
  'LastEditedBy': 1,
  'ValidFrom'

In [43]:
print(f"The person responsible for the highest comission rate is {comission[0]['FullName']} with {comission[0]['CustomFields']['CommissionRate']}")


The person responsible for the highest comission rate is Sophia Hinton with 4.55


__7. And what are the top 10 most Common Names (Primary or Surnames)?__

In [198]:
query_1 = {
    '$project': {
        '_id' : False,
        'name': {'$split': ['$FullName', ' ' ] }
    }
}

query_2 = { 
    '$unwind': "$name" } 

query_3 = {
    '$sortByCount': "$name",
}

query_4 = {
    '$match': {'_id':{'$ne': ''}}
}

query_5 = {
    '$match': {'_id':{'$ne': 'De'}}
}

query_6 = {
    '$limit': 10
}

query_7 = {
    '$project': {
        '_id':True
    }
}

pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7]

r = db.people.aggregate(pipeline)
result = list(r)

result = result[:]

names = []
for r in range(len(result)): 
    names.append(result[r]["_id"])

final_names = ', '.join(names)
print(f"The top 10 most common names are: {final_names}") 

The top 10 most common names are: Bose, Ganguly, Thakur, Roman, Van, David, Dhanishta, PrabhupÄda, Mukherjee, Bhowmick


__8. How many orders has the Customer `Tailspin Toys (Head Office)`?__

In [65]:
customerID = list(db.customers.find({"CustomerName": {'$eq': 'Tailspin Toys (Head Office)'}}))[0]['CustomerID']
customerID

1

In [66]:
TailspinCustomer = (db.orders.count_documents({"CustomerID": {'$eq': 1}}))

print(f"There are {TailspinCustomer} orders regarding customer Tailspin Toys (Head Office)")

There are 129 orders regarding customer Tailspin Toys (Head Office)


__9. How many people that have more or equal than three `OtherLanguage`?__

In [67]:
query_1 = {
    '$project':{
        '_id': False,
        'number_of_languages': { '$cond': { 'if': { '$isArray': "$OtherLanguages" }, 'then': { '$size': "$OtherLanguages" }, 'else': "NA"} 
        }
    }
}

query_2 = {
    '$match': {
        'number_of_languages':{'$gte':3}
    }
}
 

query_3 = {
    '$count': 'Number'   
}
 
    
pipeline = [query_1, query_2, query_3]

r = db.people.aggregate(pipeline)

result = list(r)

print(f"There are {result[0]['Number']} people that have more or equal than three OtherLanguage")

There are 4 people that have more or equal than three OtherLanguage


__10. Top 10 most common `OtherLanguage` for people records__

In [201]:
query_1 = {
    '$project': {
        '_id' : False,
        'other_languages': '$OtherLanguages'
    }
}

query_2 = { 
    '$unwind': "$other_languages" } 

query_3 = {
    '$sortByCount': "$other_languages",
}

query_4 = {
    '$limit': 10
}

query_5 = {
    '$project': {
        '_id':True,
    }
}

pipeline = [query_1, query_2, query_3, query_4,query_5]

r = db.people.aggregate(pipeline)
result = list(r)

languages = []
for r in range(len(result)): 
    languages.append(result[r]["_id"])

final_languages = ', '.join(languages)
print(f"The top 10 most common OtherLanguage for people records are: {final_languages}") 

The top 10 most common OtherLanguage for people records are: Finnish, Dutch, Greek, Lithuanian, Arabic, Slovak, Romanian, Croatian, Polish, Swedish


__11. Who is the most common `PickedByPersonID` person name for orders done by customer `Adriana Pena`?__

In [207]:
query_1 = {
    "$match": {"CustomerName": {"$eq": "Adriana Pena"}}
}

query_2 ={
    '$project':{
        '_id' : False,
        'CustomerName': True,
        'CustomerID': True
    }
}

query_3 = {
   '$lookup':
     {
      'from': "orders",
      'localField': "CustomerID",
      'foreignField': "CustomerID",
      'as': "inventory_docs"   
     }
}

query_4 = {
    '$unwind': "$inventory_docs"
}


query_5 = {
    '$sortByCount': "$inventory_docs.PickedByPersonID",
}

query_6 = {
   '$lookup':
     {
      'from': "people",
      'localField': "_id",
      'foreignField': "PersonID",
      'as': "people_inventory"   
     }
}

query_7 ={
    '$match':{
        '_id' : {'$gte':0}
    }
}

query_8 ={
    '$project':{
        '_id' : False,
        'count': True,
        'Name': '$people_inventory.FullName',
    }
}

query_9 = {
     '$limit': 3
}

query_10 = {
    '$project':{
        '_id' : False,
        'Name': True
    }
}

pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7, query_8, query_9, query_10]

r = db.customers.aggregate(pipeline)

result = list(r)

names = []
for r in range(len(result)): 
    names = names + (result[r]["Name"])

final_names = ', '.join(names)

print(f"The most common PickedByPersonID person names for orders done by Adriana Pena are: {final_names}") 

The most common PickedByPersonID person names for orders done by Adriana Pena are: Piper Koch, Anthony Grosse, Katie Darwin


__12. What is the average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by (`SalespersonPersonID`) person with name `Jack Potter`?__

In [202]:
query_1 = {
    "$match": {"FullName": {"$eq": "Jack Potter"}}
}

query_2 ={
    '$project':{
        '_id' : False,
        'FullName': True,
        'PersonID': True
    }
}

query_3 = {
   '$lookup':
     {
      'from': "orders",
      'localField': "PersonID",
      'foreignField': "SalespersonPersonID",
      'as': "SalesPerson_docs"   
     }
}

query_4 = {
    '$unwind': "$SalesPerson_docs"
}

query_5 ={
    '$project':{
        'FullName' : True,
        'PersonID': True,
        'dateDifference': {'$divide': [{ '$subtract': ["$SalesPerson_docs.ExpectedDeliveryDate", "$SalesPerson_docs.OrderDate"] }, 
                                        1000 * 60 * 60 * 24] } ,
    }
}

query_6 = {
    '$group': {
        '_id': {'Name' : '$FullName'}, 
        'average_days' : {'$avg' : '$dateDifference'}            
    }
}

pipeline = [query_1, query_2, query_3, query_4, query_5, query_6]

r = db.people.aggregate(pipeline)

result = list(r)

print(f"The average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by Jack Potter is: {result[0]['average_days']}")


The average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by Jack Potter is: 1.4490320833897388
